#MASTER IN DATA SCIENCE AND ENGINEERING <br>
Analytical decision support systems <br>  <br> Group #01 <br>
##Aircraft landing problem
João Pedro Pêgo, Pedro Gouveia, Ricardo Vilaça <br>
<up199502401@up.up.pt>,   <202100813@up.up.pt>  , <201308080@up.up.pt> <br><br> Faculdade de Engenharia, Universidade do Porto <br>FEUP Feb. 2022 </small>

# OR - Library
http://people.brunel.ac.uk/~mastjjb/jeb/orlib/airlandinfo.html


#Aircraft landing

There are currently 13 data files.

These data files are the test problems used in the papers "*<a href="https://www.w3schools.com/html/html_links.asp">Scheduling aircraft landings - the static case</a>*" by J.E. Beasley, M. Krishnamoorthy, Y.M. Sharaiha and D. Abramson, Transportation  Science, vol.34, 2000, pp180-197; and

"*<a href="https://www.tandfonline.com/doi/full/10.1057/palgrave.jors.2601650">Displacement problem and dynamically scheduling aircraft landings</a>*" by J.E. Beasley, M. Krishnamoorthy, Y.M. Sharaiha and
D. Abramson, Journal of the Operational Research Society, vol.55, 2004, pp54-64.

The test problems are the files:
* airland1, airland2, ..., airland13

The first eight files are those used in the Transportation Science paper referred to above.

The format of these data files is:
* number of planes (p), freeze time

for each plane i (i=1,...,p):

* appearance time, 
* earliest landing time,
* target landing time,
* latest landing time, 
* penalty cost per unit of time for landing before target,
* penalty cost per unit of time for landing after target
   
for each plane j (j=1,...p): 
* separation time required after i lands before j can land

The value of the optimal solution for each of these data
files for a varying number of runways is given in the
above papers.

The largest file is airland13 of size 800Kb (approximately). 
The entire set of files is of size 1.3Mb (approximately).

# Installs cplex

In [ ]:
!pip install cplex
!pip install docplex

# Imports libraries

In [ ]:
# imports libraries
import os
import urllib.request
import numpy as np
import pandas as pd 
import math

# import sys
try:
    import docplex.mp
except:
    raise Exception('Please install docplex. See https://pypi.org/project/docplex/')      


## Retrieves datasets

In [ ]:
# There are 13 datasets with name airlandX.txt, where X is a number in [1,13]
path = 'http://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/'
for i in range(1,14):
  filename =  'airland' + str(i) + '.txt'
  urllib.request.urlretrieve(path + filename, filename)
  os.listdir()


##Function to read the dataset

In [ ]:
#=====================================================================
# Function to fetch data from provided txt file
# @parameters- file_name : Data file(must be present in same directory)
#=====================================================================
def fetch_data(file_name):
    data=open(file_name,'r')
    lines=data.readlines()
    num_planes=int(lines[0].split()[0])
    freeze_time=int(lines[0].split()[1])

    flight_details=np.empty([num_planes,6],dtype=float)
    sep_time=np.empty([num_planes,num_planes],dtype=int)
    s=''
    for line in lines[1:]:
        s=s+line
    s=s.split()
    flag=0
    count=0
    for items in [s[x:x+6+num_planes] for x in range(0,len(s),num_planes+6)]:
        flight_details[count]=[float(x) for x in items[:6]]
        sep_time[count]=[int(x) for x in items[6:]]
        count=count+1
    # print(flight_details)
    # print(sep_time)
    data.close()
    return num_planes,flight_details,sep_time

# MIP model

## ALP - Static case - multiple runways

### Definitions:

* $P =$  the number of planes
* $R =$  the number of runways
* $E_i =$  the earliest landing time for plane $i$ ($i = 1, \ldots, P$)
* $L_i =$  the latest landing time for plane $i$ ($i = 1, \ldots, P$)
* $T_i =$   the target (preferred) landing time for plane $i$ ($i = 1, \ldots, P$)
* $S_{ij} =$  the required separation time ($\geq 0 $) between plane $i$ landing and plane $j$ landing (where plane $i$ lands before plane $j$), $i, j = 1, \ldots, P; i \neq j$
* $g_i =$  the penalty cost ($\geq 0 $) per unit of time for landing before the target time $T_i =$ for plane $i$ ($i = 1, \ldots, P$)
* $h_i =$  the penalty cost ($\geq 0 $) per unit of time for landing after the target time $T_i =$ for plane $i$ ($i = 1, \ldots, P$)
* $T_i =$  for plane $i$ ($i = 1, \ldots, P$)
* $ \delta_{ij} =$ 1 if plane $i$ lands before plane $j$, 0 otherwise ($i,j = 1, \ldots, P; i \neq j$)
* $z_{ij} =$  1  if planes $i$ and $j$ land on the same runaway, 0  otherwise ($i,j = 1, \ldots, P; i \neq j$)
* $y_{ij} =$  1  if plane $i$ ($i = 1, \ldots, P$) lands on runaway $r$, 0  otherwise  ($r = 1, \ldots, R$)

### Decision variables:

* $x_i =$ the landing time for plane $i$ ($i = 1, \ldots, P$)
* $\alpha_i =$ how soon plane $i$ ($i = 1, \ldots, P$) lands before $T_i$
* $\beta_i =$ how soon plane $i$ ($i = 1, \ldots, P$) lands after $T_i$



### Objective function:
* $ \mathrm{minimize}\sum_{i=1}^P \left( g_i \alpha_i + h_i \beta_i \right) $

### Subject to:
#### The landing time of plane $i$ must be larger than the earliest time of plane $i$:
* $ x_i \geq E_i, i = 1, \ldots, P $

#### The landing time of plane $i$ must be smaller than the latest time of plane $i$:
* $x_i \leq L_i, i = 1, \ldots, P $

#### How soon plane $i$ lands before $T_i$:
* $ \alpha_i \geq T_i - x_i, i = 1, \ldots, P $
* $ 0 \leq \alpha_i \leq  T_i - E_i, i = 1, \ldots, P $

#### How soon plane $i$ lands after $T_i$:
* $ \beta_i \geq x_i - T_i, i = 1, \ldots, P $
* $ 0 \leq \beta_i \leq  T_i - L_i, i = 1, \ldots, P $

#### Landing time:
* $ x_i = T_i - \alpha_i + \beta_i, i = 1, \ldots, P $

#### Separation time between plane $i$ and plane $j$ must be respected:
* $ x_j \geq x_i + S_{ij} z_{ij} - (L_i + S_{ij} - E_j) \delta_{ij}, i,j = 1, \ldots, P $

#### Plane $i$ can only land in one of the runways:
* $ \sum_{r=1}^{R} y_{ir} = 1, i = 1, \ldots, P $

#### If plane $i# lands in the same runway of plane $j$ the vice-versa also is true:
* $ z_{ij}=z_{ji}, i,j = 1, \ldots, P, j > i $

#### If there is any runaway $r$ for which $y_{ir}=y{jr}=1$ then $z_{ij}=1$. If $z_{ij}=0$ then the planes $i$ and $j$ cannot land on the same runaway:
* $ z_{ij} \geq y_{ir} + y_{jr} -1 , i,j = 1, \ldots, P, j > i; r, \ldots, R $

## Creates the MIP model

In [ ]:
from docplex.mp.model import Model

def static_ALP_MIP_mR(fname,R):


    # Fetch data
    P, flight_data, sep_time=fetch_data(fname)


    #Creating a CPLEX model
    name = "Aircraft Landing Problem - Static Case | MIP model | " + fname + " | Runway(s) = " + str(R) 
    mdl = Model(name)
    
    
    try:
      
        # Splits flight data
        E = flight_data[:,1]  # earliest landing time,
        T = flight_data[:,2]  # target landing time,
        L = flight_data[:,3]  # latest landing time,
        g = flight_data[:,4]  # penalty cost per unit of time for landing before target,
        h = flight_data[:,5]  # penalty cost per unit of time for landing after target

        # creates indices to use in decision variables and constrains
        ind_i = np.arange(P)
        ind_ij = []
        for i in ind_i:
            for j in ind_i:
                ind_ij.append((i,j))
        ind_ir = []
        for i in ind_i:
            for r in np.arange(R):   
                ind_ir.append((i,r))


        #Adding decision variables
        alpha   = mdl.continuous_var_dict(ind_i,lb=0,ub=mdl.infinity, name="alpha")                                                         # how soon plane i (i=1,…,P) lands before T[i]
        beta    = mdl.continuous_var_dict(ind_i,lb=0,ub=mdl.infinity, name="beta")                                                          # how soon plane i (i=1,…,P) lands after T[i]
        x       = mdl.continuous_var_dict(ind_i,lb=0,ub=mdl.infinity, name="x")                                                             # the landing time for plane i (i=1,…,P)
        delta   = mdl.binary_var_dict(ind_ij,lb=0,ub=1, name="delta")                                                                       # δij= 1 if plane i lands before plane j, 0 otherwise (i,j=1,…,P;i≠j)
        y       = mdl.binary_var_dict(ind_ir,lb=0,ub=1, name="y")                                                                           # yij= 1 if plane i (i=1,…,P) lands on runaway r, 0 otherwise (r=1,…,R)
        z       = mdl.binary_var_dict(ind_ij,lb=0,ub=1, name="z")                                                                           # zij= 1 if planes i and j land on the same runaway, 0 otherwise (i,j=1,…,P;i≠j)


        #Adding constraints
        mdl.add_constraints(x[i]>=E[i] for i in ind_i)                                                                                      # Landing time of plane i must be later than the earliest landing time
        mdl.add_constraints(x[i]<=L[i] for i in ind_i)                                                                                      # Landing time of plane i must be earlier than the before latest landing time
        mdl.add_constraints(delta[i,j]+delta[j,i]==1 for i in ind_i for j in ind_i if j!=i)                                                 # Either plane i lands before plane j or plane j lands before plane i, but not both
        mdl.add_constraints(alpha[i]>=T[i]-x[i] for i in ind_i)                                                                             # How soon plane i lands before T[i] must be larger than T[i] - x[i]
        mdl.add_constraints(beta[i]>=x[i]-T[i] for i in ind_i)                                                                              # How soon plane i lands after T[i] must be larger than x[i] - T[i]
        mdl.add_constraints(x[j]-x[i]>=sep_time[i,j]*z[j,i] - (L[i]+sep_time[i,j]-E[j])*delta[j,i] for i in ind_i for j in ind_i if j!=i)   # Separation time between plane i and plane j must be respected
        mdl.add_constraints(z[i,j]==z[j,i] for i in ind_i for j in ind_i if j>i)                                                            # If plane i lands in the same runaway as plane j, plane j lands in the same runaway as plane i
        mdl.add_constraints(mdl.sum(y[i,r] for r in np.arange(R))==1 for i in ind_i)                                                        # Plane i can only land in 1 runaway
        mdl.add_constraints(z[i,j]>=y[i,r]+y[j,r]-1 for r in np.arange(R) for j in ind_i for i in ind_i if j>i)                             # If there is any runaway r for which y[i,r]=y[j,r]=1 then z[i,j]=1. If z[i,j]=0 then the planes i and j cannot land on the same runaway 
        
        # Objective function
        total_cost = mdl.sum(alpha[i] * g[i] + beta[i] * h[i] for i in ind_i)

        # Minimize problem
        mdl.minimize(total_cost)

        # Add time limit. Apparently it is in seconds, but I could not confirm from the documentation
        mdl.set_time_limit(180)
        
        # Print information on the MIP
        mdl.print_information()

        # Solve problem and store solution
        msol = mdl.solve()
        assert msol is not None, "model can't solve"

    except AssertionError as a:
        print('Encountered an assertion error ' + str(a))

    except docplex.mp.error_handler.DOcplexException as e:
        print('Error code ' + str(e.message) + ": " + str(e))
        return e


    return P, mdl, msol

# CP model

## ALP - Static case - 1 runway

### Definitions:
* $P =$  the number of planes
* $E_i =$  the earliest landing time for plane $i$ ($i = 1, \ldots, P$)
* $L_i =$  the latest landing time for plane $i$ ($i = 1, \ldots, P$)
* $T_i =$   the target (preferred) landing time for plane $i$ ($i = 1, \ldots, P$)
* $S_{ij} =$  the required separation time ($\geq 0 $) between plane $i$ landing and plane $j$ landing (where plane $i$ lands before plane $j$), $i, j = 1, \ldots, P; i \neq j$
* $g_i =$  the penalty cost ($\geq 0 $) per unit of time for landing before the target time $T_i =$ for plane $i$ ($i = 1, \ldots, P$)
* $h_i =$  the penalty cost ($\geq 0 $) per unit of time for landing after the target time
* $T_i =$  for plane $i$ ($i = 1, \ldots, P$)

### Decision variables:
* $x_i =$ the landing time for plane $i$ ($i = 1, \ldots, P$)
* $\alpha_i =$ how soon plane $i$ ($i = 1, \ldots, P$) lands before $T_i$
* $\beta_i =$ how soon plane $i$ ($i = 1, \ldots, P$) lands after $T_i$

### Objective function:
* $ \mathrm{minimize}\sum_{i=1}^P \left( g_i \alpha_i + h_i \beta_i \right)$

### Subject to:
#### The landing time of plane $i$ must be larger than the earliest time of plane $i$:
* $ x_i \geq E_i, i = 1, \ldots, P $

#### The landing time of plane $i$ must be smaller than the latest time of plane $i$:
* $x_i \leq L_i, i = 1, \ldots, P $

#### How soon plane $i$ lands before $T_i$:
* $ \alpha_i = T_i - x_i, i = 1, \ldots, P $

#### How soon plane $i$ lands after $T_i$:
* $ \beta_i = x_i - T_i, i = 1, \ldots, P $

#### Separation time between plane $i$ and plane $j$ must be respected:
* $ x_j \geq x_i + S_{ij} $ if $i$ lands before $j$


## Creates the CP model -  1 runway

In [ ]:
from docplex.cp.model import CpoModel

def static_ALP_CP_1R(fname):

    # Fetch data
    P, flight_data, sep_time=fetch_data(fname)

    #Creating a CPLEX model
    name = "Aircraft Landing Problem - Static Case | CP model | " + fname + " | Runway(s) = 1" 
    mdl = CpoModel(name)
    sep_time
    
    try:
      
        # Splits flight data
        E = flight_data[:,1]  # earliest landing time,
        T = flight_data[:,2]  # target landing time,
        L = flight_data[:,3]  # latest landing time,
        g = flight_data[:,4]  # penalty cost per unit of time for landing before target,
        h = flight_data[:,5]  # penalty cost per unit of time for landing after target

        # creates indices to use in decision variables and constrains
        ind_i = np.arange(P)

        #
        max_INT = int(max(L[i]-E[i] for i in ind_i))
        min_E = int(min(E))
        max_L = int(max(L))
        max_cost = max(max(g),max(h))

        #Adding decision variables
        alpha   = mdl.integer_var_list(size=P, min=0, max=max_INT, name='alpha')                    # how soon plane i (i=1,…,P) lands before T[i]
        beta    = mdl.integer_var_list(size=P, min=0, max=max_INT, name='beta')                     # how soon plane i (i=1,…,P) lands after T[i]
        x       = mdl.integer_var_list(size=P, min=min_E, max=max_L, name='x')                      # the landing time for plane i (i=1,…,P)

        #Adding constraints
        mdl.add(x[i] >= E[i] for i in ind_i)                                                        # Landing time of plane i must be later than the earliest landing time
        mdl.add(x[i] <= L[i] for i in ind_i)                                                        # Landing time of plane i must be earlier than the before latest landing time
        mdl.add(alpha[i] == mdl.max(0, T[i] - x[i]) for i in ind_i)                                 # how soon plane i (i=1,…,P) lands before T[i]
        mdl.add(beta[i] == mdl.max(0, x[i] - T[i]) for i in ind_i)                                  # how soon plane i (i=1,…,P) lands after T[i]

        for i in ind_i:             
            for j in range(i+1, P):
                mdl.add(mdl.logical_or(x[j]-x[i] >= sep_time[i,j] , x[i]-x[j] >= sep_time[j,i]))    # separation time betwen plane i and plane j

        
        # Objective function
        total_cost = mdl.integer_var(min=0, max=int(max_cost*(max_L-min_E)), name='total_cost')
        mdl.add(total_cost == mdl.sum(alpha[i] * g[i] + beta[i] * h[i] for i in ind_i))


        # Minimize problem
        mdl.add(mdl.minimize(total_cost))
      
        # Print information on the CP
        mdl.print_information()

        # Solve problem and store solution
        msol = mdl.solve(TimeLimit=180)

        objective = msol.get_value(total_cost)

        assert msol is not None, "model can't solve"

    except AssertionError as a:
        print('Encountered an assertion error ' + str(a))
        return a

    except docplex.mp.error_handler.DOcplexException as e:
        print('Error code ' + str(e.message) + ": " + str(e))
        return e


    return P, mdl, msol, objective

# CP model

## ALP - Static case - multiple runways

### Definitions:
* $P =$  the number of planes
* $R =$  the number of runways
* $n_{slots} = \mod(P/R) + 1 $  the number of slots in each runway
* $fP = n_{slots} \cdot R - P$  the number of fictitious planes
* $nP = P + fP$  the total number of  planes
* $E_i =$  the earliest landing time for plane $i$ ($i = 1, \ldots, nP$)
* $L_i =$  the latest landing time for plane $i$ ($i = 1, \ldots, nP$)
* $T_i =$   the target (preferred) landing time for plane $i$ ($i = 1, \ldots, nP$)
* $S_{ij} =$  the required separation time ($\geq 0 $) between plane $i$ landing and plane $j$ landing (where plane $i$ lands before plane $j$), $i, j = 1, \ldots, P; i \neq j$
* $g_i =$  the penalty cost ($\geq 0 $) per unit of time for landing before the target time $T_i =$ for plane $i$ ($i = 1, \ldots, nP$)
* $h_i =$  the penalty cost ($\geq 0 $) per unit of time for landing after the target time $T_i =$ for plane $i$ ($i = 1, \ldots, nP$)
* $T_i =$  for plane $i$ ($i = 1, \ldots, nP$)
* The values of $S_{ij}$ are the same as for the MILP for the real planes ($i,j = 1, \ldots, P$) and 0 for the fictitious planes ($i,j = P +1 , \ldots, nP$).
* $E_i = min_E = \min(E_i | i = 1, \ldots, P)$ for ($i = P + 1, \ldots, nP$)
* $L_i = max_L = \max(L_i | i = 1, \ldots, P)$ for ($i = P + 1, \ldots, nP$)
* $T_i =\frac{max_L + min_E}{2}$ for ($i = P + 1, \ldots, nP$)
* $g_i = h_i = 0$ for ($i = P + 1, \ldots, nP$) The cost of the ficticious is zero so they don't affect the objective function
* $SP_s= S_{ij}$, where $s= i + nP \cdot j$ 1D array with the values of $S_{ij}$
  
### Decision variables:
* $slot_i =$ the slot where plane $i$ lands ($i = 1, \ldots, nP$)
* $t_{land, i} =$ the landing time for plane $i$ ($i = 1, \ldots, nP$)
* $\alpha_i =$ how soon plane $i$ ($i = 1, \ldots, nP$) lands before $T_i$
* $\beta_i =$ how soon plane $i$ ($i = 1, \ldots, nP$) lands after $T_i$

### Objective function:
* $ \mathrm{minimize}\sum_{i=1}^P \left( g_i \alpha_i + h_i \beta_i \right)$

### Subject to:
#### The landing time of plane $i$ must be larger than the earliest time of plane $i$:
* $ x_i \geq E_i, i = 1, \ldots, P $

#### The landing time of plane $i$ must be smaller than the latest time of plane $i$:
* $x_i \leq L_i, i = 1, \ldots, P $

#### How soon plane $i$ lands before $T_i$:
* $ \alpha_i = T_i - x_i, i = 1, \ldots, P $

#### How soon plane $i$ lands after $T_i$:
* $ \beta_i = x_i - T_i, i = 1, \ldots, P $

#### Separation time between plane $i$ and plane $j$ must be respected:
* $ x_j \geq x_i + S_{ij} $ if $i$ lands before $j$


## Creates the CP model -  multiple runways

In [ ]:
from docplex.cp.model import CpoModel
import numpy as np


# Function to add false planes (lines and columns) to flight data and separation times
def falseplanes(array, new_columns, new_rows):
    num_rows, num_cols = array.shape
    azeros=np.zeros((num_rows, new_columns))
    array= np.c_['r',array, azeros]

    azeros=np.zeros((new_rows, num_cols + new_columns))
    array= np.r_['c',array, azeros]

    return array


    
def static_ALP_CP_mR(fname,R):


    # Fetch data
    P, flight_data, sep_time=fetch_data(fname)

    #Creating a CPLEX model
    name = "Aircraft Landing Problem - Static Case | CP model | " + fname + " | Runway(s) = " + str(R) 
    mdl = CpoModel(name)
        
    try:
      
        # global parameters
        max_INT = int(max(flight_data[i,3]- flight_data[i,1] for i in range(P)))
        min_E = int(min(flight_data[:,1]))
        # max_E = int(max(flight_data[:,1]))
        # min_L = int(min(flight_data[:,3]))
        max_L = int(max(flight_data[:,3]))
        max_g = int(max(flight_data[:,4]))
        max_h = int(max(flight_data[:,5]))  
        max_SP = int(np.amax(sep_time))
        
        max_cost = max(max_g,max_h)

        nslots = int(P/R) + 1
        # nslots = int(math.ceil(P / R)) 
        # nslots = P

        fP = (nslots * R) - P                               # number of ficticious planes to create

        nP = P + fP                                         # number of planes + number of ficticious planes
       
        # creates arrays for false planes which have zero costs and a time landing range equal to set of planes
        new_st = falseplanes(sep_time, fP, fP)
        new_fd = falseplanes(flight_data, fP, fP)

        # convert to integers
        new_st=new_st.astype(int)
        new_fd=new_fd.astype(int)
         


        # Splits flight data
        E_ = new_fd[:,1]  # earliest landing time
        T_ = new_fd[:,2]  # target landing time
        L_ = new_fd[:,3]  # latest landing time
        g_ = new_fd[:,4]  # penalty cost per unit of time for landing before target
        h_ = new_fd[:,5]  # penalty cost per unit of time for landing after target

        
        # the time windows of the false planes correspond to the maximum and minimumm values of all true planes. The target is the average of the interval
        for i in range (P, P + fP  ):
            E_[i] = min_E
            L_[i] = max_L
            T_[i] = int((min_E + max_L)/2)
        

        #Adding decision variables
        # E = mdl.integer_var_list(nP, min=min_E, max=max_E, name='E')                # earliest landing time
        T = mdl.integer_var_list(nP, min=min_E, max=max_L, name='T')                # target landing time
        # L = mdl.integer_var_list(nP, min=min_L, max=max_L, name='L')                # latest landing time
        g = mdl.integer_var_list(nP, min=0, max=max_g, name='g')                    # penalty cost per unit of time for landing before target
        h = mdl.integer_var_list(nP, min=0, max=max_h, name='h')                    # penalty cost per unit of time for landing after target
        SP = mdl.integer_var_list(nP*nP, min=0, max=max_SP, name='SP')              # array with the planes separation times
        alpha   = mdl.integer_var_list(nP, min=0, max=max_INT, name='alpha')        # how soon plane i (i=1,…,P) lands before T[i]
        beta    = mdl.integer_var_list(nP, min=0, max=max_INT, name='beta')         # how soon plane i (i=1,…,P) lands after T[i]
        slot   = mdl.integer_var_list(nP, min=0, max=nP-1, name='slot')             # array with the landing slots in all runways. The values inside the slots will be the plane numbers
        

        t_land=[]
        
        for i in range(nP):
            mdl.add(mdl.integer_var(min = int(E_[i]), max= int(L_[i]), name='x' + str(i)))
            t_land.append("t_land_" + str(i))

        t_land = mdl.integer_var_list(nP, min=min_E, max=max_L, name='t_land')           # array with the landing times of each plane (real and ficticious)

        mdl.add(mdl.all_diff(slot))

        # mdl.add(mdl.expr_list() t_land)
        #Adding constraints
     
        # copies the values of the flighs data to arrays of the model, so they can be acessed inside the model dynamically
        for i in range(nP):
            # mdl.add(E[i] == int(E_[i]))
            mdl.add(T[i] == int(T_[i]))
            # mdl.add(L[i] == int(L_[i]))
            mdl.add(g[i] == int(g_[i]))
            mdl.add(h[i] == int(h_[i]))

        for i in range(nP):
            for j in range(nP):
                mdl.add(SP[i + nP * j] == int(new_st[i,j]))
        
        
        mdl.export_model('model.txt')
        # mdl.add(mdl.element(E, slot[i]) <= mdl.element(t_land, slot[i]) for i in range(nP))                            # Landing time of plane i must be later than the earliest landing time
        # mdl.add(mdl.element(L, slot[i]) >= mdl.element(t_land, slot[i]) for i in range(nP))                            # Landing time of plane i must be earlier than the before latest landing time
        mdl.add(alpha[i] == mdl.max(0, mdl.element(T, slot[i]) - mdl.element(t_land, slot[i])) for i in range(nP))     # how soon plane i (i=1,…,P) lands before T[i]
        mdl.add(beta[i] == mdl.max(0, mdl.element(t_land, slot[i]) - mdl.element(T, slot[i]))  for i in range(nP))     # how soon plane i (i=1,…,P) lands after T[i]

        # separation time between planes
        for r in range(R):
            for i in range(nslots):
                for j in range( i + 1, nslots):
                    mdl.add(mdl.element(SP, slot[i + nslots * r] + nP * slot[j + nslots * r]) <= (mdl.element(t_land, slot[j+ nslots * r]) - mdl.element(t_land, slot[i+ nslots * r])))




        #http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.model.py.html#docplex.cp.model.CpoModel.set_starting_point
        
        
        # stp = mdl.create_empty_solution()
        
        # for r in range(R):
        #     for i in range(nslots):
        #         stp[slot[i]]
                
        # stp_slot = [r + i * R for r in range(R) for i in range(nslots)]
        # stp_slot = tuple(stp_slot)
        # stp_t_land =  T

        # mdl.add_integer_var_solution(stp_slot)

        # # Objective function
        total_cost = mdl.integer_var(min=0, max=int(max_cost*(max_L-min_E)), name='total_cost')
        mdl.add(total_cost == mdl.sum(alpha[i] * mdl.element(g, slot[i]) + beta[i] * mdl.element(h, slot[i]) for i in range(nP)))

        # Minimize problem
        mdl.add(mdl.minimize(total_cost))
      
        # Print information on the CP
        mdl.print_information()
    

        # Solve problem and store solution
        msol = mdl.solve(TimeLimit=180)

        objective = msol.get_value(total_cost)

        assert msol is not None, "model can't solve"

    except AssertionError as a:
        print('Encountered an assertion error: ' + str(a))
        return a

    except docplex.mp.error_handler.DOcplexException as e:
        print('Error code ' + str(e.message) + ": " + str(e))
        return e


    return P, mdl, msol, objective

# Runs a single problem (MIP)

In [ ]:
filename = 'airland3.txt'
r = 2
P_, mdl_, msol_ = static_ALP_MIP_mR(filename, r)

mdl_.report()
#mdl_.print_solution()

# Runs a single problem (CP_1R)

In [ ]:
filename = 'airland3.txt'

P_, mdl_, msol_, objective_= static_ALP_CP_1R(filename)


#msol_.print_solution()

print('Objective value: ' + str(objective_))


# Runs a single problem (CP_mR)

In [ ]:
filename = 'airland1.txt'
r = 2
P_, mdl_, msol_, objective_= static_ALP_CP_mR(filename, r)


# msol_.print_solution()

print('Objective value: ' + str(objective_))

print('sol_time: ' + str(round(msol_.solveTime,3)))


# Loops all the files and writes KPI to file

In [ ]:
def loop_files(fstart, fend, nrunaways):

  import pandas as pd 

  # creates empty list where the rows of the df will be stored
  rows_list1 = []
  rows_list2 = []
  
  # loop all files
  for i in range(fstart, fend + 1):
    # file to open
    filename =  'airland' + str(i) + '.txt'

    # loop over range of runways number
    for r in range(1, nrunaways + 1):
      try: 
        P_MIP, mdl_MIP, msol_MIP = static_ALP_MIP_mR(filename, r)
      
        # dictionary with the row data
        dict1 = {'Case': i ,'R': r, 'P': P_MIP, 'n_cont_var' : mdl_MIP.number_of_continuous_variables , 'n_bin_var' : mdl_MIP.number_of_binary_variables , 'n_const' : mdl_MIP.number_of_constraints , 'value' : int(round(mdl_MIP.objective_value, 0)) , 'sol_time' : round(mdl_MIP.solve_details.time,3)} 
        # appends to list of dictionary
        rows_list1.append(dict1)

      except:
        dict1 = {'Case': i ,'R': r}
        rows_list1.append(dict1)


    # loop over range of runways number
    
    r = 1
    try: 
      P_CP, mdl_CP, msol_CP, objective_CP = static_ALP_CP_1R(filename)
      
      # dictionary with the row data
      dict2 = {'Case': i ,'R': r, 'P': P_CP, 'value' : int(round(objective_CP,0)) , 'sol_time' : round(msol_CP.solveTime,3)} 
      # appends to list of dictionary
      rows_list2.append(dict2)

    except:
      dict2 = {'Case': i ,'R': r}
      rows_list2.append(dict2)

    # creates dataframe with the list of dictionaries
    df_MIP = pd.DataFrame(rows_list1)
    df_CP = pd.DataFrame(rows_list2)  

  return df_MIP, df_CP

fstart = 1
fend = 8
nrunaways = 5

df_results_MIP, df_results_CP = loop_files(fstart, fend, nrunaways)

# writes the results to a csv file
df_results_MIP.to_csv('solutions_MIP.csv')
df_results_CP.to_csv('solutions_CP.csv')

In [ ]:
df_results_MIP.head()

In [ ]:
df_results_CP.head()

In [ ]:
# Merge the two data frames
df_merge = df_results_MIP.merge(df_results_CP[['Case','R','value','sol_time']], how = 'outer', on = ['Case', 'R'] )

df_merge.columns = ['Case', 'R', 'P', 'n.cvar', 'n.bvar', 'n.const', 'value1',
       'sol.time1', 'value2', 'sol.time2']
df_merge.head()


In [ ]:
print(df_merge.to_latex(na_rep = '--' , index=False))
